**Tutorial 7 - Photometric redshifts for QSOs?  Linear regression with scikit-learn**

In this tutorial we will try to build a model to predict QSO redshifts from their photometry in 5 bands.

We will get some practice with the scikit-learn package which contains powerful tools for machine learning problems.  It is well worth getting to know the capabilities of these tool (https://scikit-learn.org/stable/user_guide.html).

In [129]:
import numpy as np
import matplotlib.pyplot as plt


1) Import the data from the file SDSS_QSO.dat.  This file contains data from the Slone Digital Sky Survey on QSO's.  It has the name, redshift, photometry in 5 bands and errors, the brightness in the X-ray, and brightness in radio.

Further information on columns:
FIRST - brightness in radio in magnitudes, 0 is undetected, -1 not observed
ROSAT - X-ray brightness in log(counts), -9 not detected

2) Make numpy vectors of the magnitudes in the 5 bands.  Then put these into a block that is shape [n_bands,n_qso].  Also make a vector of the redshifts.

3) From sklearn import linear_model.  Make a linear_model.LinearRegression() object 
and then fit a model that predicts redshifts from the colors. (see https://scikit-learn.org/stable/modules/linear_model.html for more information.)  Print out the coefficients.  Use the model.score() function to give the score for the model which in this case is the R^2 statistic (coefficient of determination).

In [2]:
from sklearn import linear_model
linear_mod = linear_model.LinearRegression()

.
.
.

4) Use the model to predict the redshifts for all of the QSOs.  Make a scatter plot of the predicted vs observed redshifts. Decreasing the alpha parameters can make this clearer.

5) Scatter plots can be deceiving.  The density of points can be estimated with a Gaussian kernel.  This should do that.  Put some labels on the plot and overlay a contour plot.

In [ ]:
from scipy.stats import kde

nbins = 40
k = kde.gaussian_kde(np.array([predictions,observations]))

xi, yi = np.mgrid[0:5:nbins*1j, 0:5:nbins*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))
 
plt.pcolormesh(xi,yi, zi.reshape(xi.shape), shading='gouraud')
plt.plot([0,5],[0,5],linestyle=':')
plt.colorbar()

plt.show()

6) Do a k-fold validation with k=10 using sklearn.model_selection.KFold. Make an array of the MSE's for each resampling.  Make a histogram of the square root of the MSE's divided by the standard deviation of the redshifts.  This is a measure of how well the prediction model works compared to randomly sampling from the redshift distribution.  

Is this a good prediction model?

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

kf = KFold(n_splits = k)
.
.
for train_index, test_index in kf.split(redshift):
  .

.

7) Do ridge regression on the same data. The strength of the prior is controlled by the alpha parameter.  linear_model.RidgeCV() will automatically select an alpha from the ones provided by doing cross-validation for each one and taking the one with the best MSE. Use alphas=[0.1, 1.0, 10.0, 100.0,1000.0] and cv=5.  What is the best alpha? Print out the coefficients.  Print the score.  Make a scatter plot of the predictions vs the observed redshifts.

8) Do LASSO regression using linear_model.LassoCV() and output the same thing as for ridge regression.  Which single color best predicts the redshift.

9) Let us include terms that are higher order in the magnitudes.  Use the function 
PolynomialFeatures(3).fit_transform() to extend the color matrix to one with extra columns that have powers and products of the existing columns up to third order.
Then fit the ridge regression model found above using this feature matrix.
What is the score for this model?

In [4]:
from sklearn.preprocessing import PolynomialFeatures


10) Plot the predicted vs the observed redshifts for this model as a scatter plot and a contour plot like was done in 5).